In [3]:
import os

import numpy as np
import matplotlib.pyplot as plt

import torch
from torch import nn
from torch.nn import Parameter
import torch.nn.functional as F
from torch.nn.modules import loss
import torch.optim as optim
from torch.autograd import Variable
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision import transforms
from six.moves.urllib.request import urlretrieve
import csc413_final_project_code
from csc413_final_project_code import DCGenerator
from urllib.error import URLError
from urllib.error import HTTPError
from torch.utils.tensorboard import SummaryWriter
import numpy as np
import pandas as pd

def log_to_tensorboard(iteration, losses):
    writer = SummaryWriter("/content/runs/")
    for key in losses:
        arr = losses[key]
        writer.add_scalar(f'loss/{key}', arr[-1], iteration)
    writer.close()


class AttrDict(dict):
    def __init__(self, *args, **kwargs):
        super(AttrDict, self).__init__(*args, **kwargs)
        self.__dict__ = self


def to_var(tensor, cuda=True):
    """Wraps a Tensor in a Variable, optionally placing it on the GPU.
        Arguments:
            tensor: A Tensor object.
            cuda: A boolean flag indicating whether to use the GPU.
        Returns:
            A Variable object, on the GPU if cuda==True.
    """
    if cuda:
        return Variable(tensor.cuda())
    else:
        return Variable(tensor)

data/emojis.tar.gz
Extracting file.
                                      Opts                                      
--------------------------------------------------------------------------------
                             image_size: 32                                     
                             g_conv_dim: 32                                     
                             d_conv_dim: 64                                     
                             noise_size: 100                                    
                            train_iters: 100000                                 
                                      X: letters                                
                                     lr: 0.0005                                 
                                  beta1: 0.5                                    
                                  beta2: 0.999                                  
                             batch_size: 64                              

In [4]:
def get_zero_vector(dim):
    """
    """
    return torch.zeros(1, dim).requires_grad_()


def get_emnist_loader(opts):
    transform = transforms.Compose([
                    transforms.Resize(opts.image_size),
                    transforms.ToTensor(),
                    transforms.Normalize((0.5), (0.5)),
                ])
    train = datasets.EMNIST(".", split=opts.X,train = True, download = True, transform= transform)
    test = datasets.EMNIST(".", split=opts.X,train = False, download = True, transform = transform)
    
    train_dloader = DataLoader(dataset=train, batch_size=opts.batch_size, shuffle=True,num_workers=opts.num_workers, drop_last=True)
    
    test_dloader = DataLoader(dataset=test, batch_size=opts.batch_size, shuffle=False,num_workers=opts.num_workers)
    # return train_dloader, test_dloader
    return test_dloader

In [5]:
def load_generator(opts) -> nn.Module:
    """
    """
    G_path = os.path.join(opts.load, opts.G_name)
    print(G_path)
    G = DCGenerator(noise_size=opts.noise_size,
                    conv_dim=opts.g_conv_dim, spectral_norm=False)

    G.load_state_dict(torch.load(
        G_path, map_location=lambda storage, loc: storage))
    return G


def train(opts):
    G = load_generator(opts)
    z = get_zero_vector(opts.noise_size)

    z_optimizer = optim.SGD([z], lr=1e-4)
    test_data = get_emnist_loader(opts)
    train_iter = iter(test_data)
    iter_per_epoch = len(train_iter)

    criteria = nn.MSELoss()
    iteration = 0
    epoch = 0
    losses = {"reconstruction_loss": []}
    for i in range(opts.iterations):
        sample, target = train_iter.next()
        if iteration % iter_per_epoch == 0:
            epoch += 1
            train_iter = iter(test_data)
            print("epoch:", epoch)

        z_optimizer.zero_grad()
        Loss = criteria(G(z.unsqueeze(2).unsqueeze(3)), sample)
        Loss.backward()
        z_optimizer.step()
        if iteration % opts.log_step == 0:
            losses["reconstruction_loss"].append(Loss.item())
            log_to_tensorboard(iteration, losses)
            print('iteration: ', iteration, "loss: ", Loss.item())
            
            df = pd.DataFrame(losses) 
            df.to_csv('reconstruction_losses.csv') 

        iteration += 1

In [7]:
args = AttrDict()
args_dict = {
        'image_size': 32,
        'g_conv_dim': 32,
        'noise_size': 100,
        'num_workers': 0,
        'iterations': 120000,
        'X': 'letters',
        'batch_size': 16,
        'load': "/content/pretrained_models/",
        'log_step': 1000,
        'sample_every': 200,
        'checkpoint_every': 1000,
        'G_name': 'G-100.pkl'
    }
args.update(args_dict)
train(args)

/content/pretrained_models/G-100.pkl



Extracting ./EMNIST/raw/emnist.zip to ./EMNIST/raw
Processing byclass


/usr/local/lib/python3.7/dist-packages/torchvision/datasets/mnist.py:502: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:143.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


Processing bymerge
Processing balanced
Processing letters
Processing digits
Processing mnist
Done!
epoch: 1


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/loss.py:528: UserWarning: Using a target size (torch.Size([16, 1, 32, 32])) that is different to the input size (torch.Size([1, 1, 32, 32])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


iteration:  0 loss:  0.6260254383087158


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/loss.py:528: UserWarning: Using a target size (torch.Size([16, 1, 32, 32])) that is different to the input size (torch.Size([1, 1, 32, 32])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


iteration:  1000 loss:  0.3870023787021637
epoch: 2
iteration:  2000 loss:  0.6174291372299194
epoch: 3
iteration:  3000 loss:  0.46050530672073364
epoch: 4
iteration:  4000 loss:  0.5706568956375122
iteration:  5000 loss:  0.5679577589035034
epoch: 5
iteration:  6000 loss:  0.47721099853515625
epoch: 6
iteration:  7000 loss:  0.5574061870574951
epoch: 7
iteration:  8000 loss:  0.5497928857803345
iteration:  9000 loss:  0.37520188093185425
epoch: 8
iteration:  10000 loss:  0.5386103391647339
epoch: 9
iteration:  11000 loss:  0.36219146847724915
epoch: 10
iteration:  12000 loss:  0.42995479702949524
epoch: 11
iteration:  13000 loss:  0.5012283325195312
iteration:  14000 loss:  0.38744527101516724
epoch: 12
iteration:  15000 loss:  0.6068238615989685
epoch: 13
iteration:  16000 loss:  0.46048253774642944
epoch: 14
iteration:  17000 loss:  0.5723498463630676
iteration:  18000 loss:  0.5656113624572754
epoch: 15
iteration:  19000 loss:  0.47531357407569885
epoch: 16
iteration:  20000 loss: